In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Adam
import torch.nn.functional as F

import csv
from skimage import io
from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchsummary import summary

import matplotlib.pyplot as plt
import matplotlib as mpl
import time
import os
import pathlib
import copy
from datetime import date

import import_ipynb
import ResNetCaps_E
import MiniBatch_generator
import losses
import CHIMP_DataLoader

ATET_use = False
LFW_use = True
CHIM_use = False

load_model = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

importing Jupyter notebook from ResNetCaps_E.ipynb
importing Jupyter notebook from MiniBatch_generator.ipynb
importing Jupyter notebook from losses.ipynb
importing Jupyter notebook from ohem.ipynb
importing Jupyter notebook from CHIMP_DataLoader.ipynb


In [2]:
#DATASET INITIALIZATION -----------------------------------------------------
if ATET_use:
    folder = 'ATET'
    dataset_folder = "ATeT_faces/orl_faces/"
if LFW_use:
    folder = 'LFW'
    dataset_folder = "lfw/"
if CHIM_use:
    folder = 'CHIM'
    dataset_folder = "chimpanzee_faces-master/datasets_cropped_chimpanzee_faces/data_CZoo/annotations_czoo.txt"   
    

dataset_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),        
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

dataset_folder = os.path.join("/home/rita/JupyterProjects/EYE-SEA/DataSets/Verification", dataset_folder)


In [3]:
if CHIM_use:
    print('Loading chimp dataset')
    dataset = CHIMP_DataLoader.Chimp_Dataset(dataset_folder,dataset_transform,0.8)
    dataset_t = CHIMP_DataLoader.Chimp_Dataset(dataset_folder,dataset_transform,0.8,train=False)
else:
    print('Loading {} dataset'.format(folder))
    dataset = MiniBatch_generator.mini_batch(dataset_folder, dataset_transform,0.2)

Loading LFW dataset


In [4]:
#MODEL LOADER ---------------------------------- 

PATH= os.path.join(os.getcwd(),os.path.join('Log_model/HAP2S',folder,'DIGIT/',(date.today()).isoformat()))
pathlib.Path(PATH).mkdir(parents=True, exist_ok=True)
if len(os.listdir(PATH)) > 2 and load_model:
    print('Loading model from PATH: {}'.format(PATH))
    model = ResNetCaps_E.ResNetCaps_E(DigitEnd=False)
    if pick_model == -1:
        init = len(os.listdir(PATH))-2        
        model.load_state_dict(torch.load(os.path.join(PATH,str(init))))
    else:
        model.load_state_dict(torch.load(os.path.join(PATH,str(pick_model))))
        init = pick_model
    model.eval()
else:
    print('Creating a new model')
    init=0
    model = ResNetCaps_E.ResNetCaps_E()

selected = 0

Creating a new model


In [5]:
model = ResNetCaps_E.ResNetCaps_E(DigitEnd=True)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)
    
model = model.to(device)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr = 0.001)
criterion = losses.HAP2STripletLoss()

In [7]:
#TRAINING PHASE ----------------------------------minibatch random everyepoch

n_epochs = 100
loss_list_b = []

for epoch in range(n_epochs): 
    print('epoch {}:{}'.format(epoch+1, n_epochs)) 
    model.train()
    loss_collect = 0
    in_a,labels = dataset.prepare_batch()
    in_a = torch.stack(in_a)
    in_a = in_a.to(device)
    labels = torch.Tensor(labels).to(device)

    #Compute embeddings for anchor, positive, and negative images

    emb_a = model(in_a)
    emb_a = emb_a.view(in_a.size(0),-1)
    optimizer.zero_grad()
    loss = criterion(emb_a.squeeze(),labels)
    loss_collect +=loss.item()

    print("lost per batch {}".format(loss))

    loss.backward()
    optimizer.step()
    loss_list_b.append(loss)
    torch.save(model.state_dict(), os.path.join(PATH,str(epoch)))

epoch 1:100
torch.Size([846, 846])


ResNetCaps_E.ipynb:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  "from torch.utils.data import Dataset, DataLoader\n",


RuntimeError: shape '[846, -1]' is invalid for input of size 16930

In [ ]:
print(in_a.size())
print(emb_a.size())

In [ ]:
epochs = np.arange(1,n_epochs+1)
plt.plot(epochs, loss_list_b, color='pink')
plt.xlabel('epochs')
plt.ylabel('lost')
plt.title('Training phase')
plt.show() 

In [ ]:
def accuracy(feats,labels):
    D = losses.euclidean_distance(feats,feats)
    N = D.size(0)
    print(N)
    # shape [N, N]
    is_pos = labels.expand(N, N).eq(labels.expand(N, N).t())
    is_neg = labels.expand(N, N).ne(labels.expand(N, N).t())
    
    # Exclude selfs for positive samples
    device = labels.device
    v = torch.zeros(N).to(device).type(is_pos.dtype)
    mask = torch.diag(torch.ones_like(v)).to(device).type(is_pos.dtype)
    is_pos = mask * torch.diag(v) + (1. - mask) * is_pos

    # `dist_ap` means distance(anchor, positive)
    dist_ap = D[is_pos].contiguous().view(N, -1)
    # `dist_an` means distance(anchor, negative)
    dist_an = D[is_neg].contiguous().view(N, -1)
    
    threshold = (torch.mean(dist_ap) + torch.mean(dist_an))/2
    
    positives_True =  0
    for i in dist_ap:
        for j in range(len(i)):
            if i[j].item() < threshold: positives_True += 1 
    negatives_True =  0
    for i in dist_an:
        for j in range(len(i)):
            if i[j].item() > threshold: negatives_True += 1   
    
    VAL = positives_True/dist_ap.numel()
    FAR = negatives_True/dist_an.numel()
    
    return positives_True, negatives_True, VAL, FAR, threshold

In [ ]:
#Test
#test_minibatch 

in_a,labels = dataset.prepare_batch_test()
in_a = torch.stack(in_a)
in_a = in_a.to(device)
labels = torch.Tensor(labels).to(device)
emb_a = model(in_a)
loss = criterion(emb_a.squeeze(),labels)
P_T, N_T, VAL, FAR,th = accuracy(emb_a.squeeze(),labels)
print("Loss {}. Threshold {}: P_T {} N_T {} VAL {} FAR {}".format(loss, th, P_T, N_T, VAL, FAR))



In [ ]:
torch.cuda.empty_cache()